In [ ]:
!pip install transformers -q

## Imports

In [ ]:
import tensorflow as tf
import torch
import time
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from transformers import BertTokenizer
from torch.utils.data import Dataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader
import torch.optim as optim
from datetime import datetime
import torch.nn.functional as F

In [ ]:
data_path = '/content/bert_ner_moountain_dataset.csv'
df = pd.read_csv(data_path, delimiter=";",  error_bad_lines=False )

<ipython-input-3-dc66f1f9c607>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(data_path, delimiter=";",  error_bad_lines=False )
Skipping line 1726: expected 2 fields, saw 3
Skipping line 3338: expected 2 fields, saw 3



In [ ]:
df.sample(5)

,label,sentence
1964,O,The neighborhood picnic was filled with laught...
721,Mountain,The Shoshone Mountains in Nevada are a hidden ...
3372,O,The success of a trek lies in the realization ...
2095,O,The cozy fireplace at the cabin provided warmt...
2993,O,Is there anything more liberating than the fee...


In [ ]:
df.columns

Index(['label', 'sentence'], dtype='object')

In [ ]:
df['label'].unique()

array(['O', 'Mountain'], dtype=object)

In [ ]:
df['label'] = df['label'].map(lambda x: x.strip())

In [ ]:
df['label'].unique()

array(['O', 'Mountain'], dtype=object)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenized_inputs = tokenizer(df["sentence"].tolist(), padding=True, truncation=True, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, tokenized_inputs, labels):
        self.tokenized_inputs = tokenized_inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokenized_inputs["input_ids"][idx],
            "attention_mask": self.tokenized_inputs["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx])
        }

In [ ]:
dataset = CustomDataset(tokenized_inputs, labels=df["label"].map({"Mountain": 1, "O": 0}).tolist())

In [ ]:
dataset[0]

{'input_ids': tensor([  101,  2298,  2012,  2216, 14863, 13639,  2890,  8483,  1999,  1996,
          3292,  1010,  2027,  1005,  2128,  5621,  3052, 17904,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 'labels': tensor(0)}

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def train(num_epochs=2):
  """
  [num_epochs] 2-4 is recomended for fine-tuning bert
  """
  dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

  optimizer = optim.AdamW(model.parameters(), lr=2e-5)
  criterion = torch.nn.CrossEntropyLoss()

  for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      start_time = time.time()
      for batch in dataloader:

          inputs = batch["input_ids"]
          attention_mask = batch["attention_mask"]
          labels = batch['labels']
          outputs = model(inputs, attention_mask=attention_mask)
          loss = criterion(outputs.logits, labels)
          total_loss += loss.item()

          loss.backward()
          optimizer.step()
      end_time = time.time()
      epoch_time = end_time - start_time

      average_loss = total_loss / len(dataloader)
      print(f"Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss}  \nTime: {epoch_time} seconds")

In [ ]:
"""
Epoch 1/3 - Average Loss: 0.4773173794016108
Time: 1612.7836396694183 seconds
Epoch 2/3 - Average Loss: 0.6529922951180656
Time: 1605.6941952705383 seconds
Epoch 3/3 - Average Loss: 0.9067454332703943
Time: 1613.9223237037659 seconds
"""
train()



Epoch 1/2 - Average Loss: 0.6942762351250863  
Time: 1615.6793551445007 seconds
Epoch 2/2 - Average Loss: 0.7359184719420768  
Time: 1614.4291133880615 seconds


In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
torch.save(model.state_dict(), '/content/model.pth')

In [ ]:
def predict_sentence(sentence_x, model, tokenizer):
    """
      [sentence_x] - String sentence to predict
      [model] - BERT model that will predict on sentence
      [tokenizer] - tokenizer used for that model
    """
    tokenized_inputs = tokenizer(sentence_x.split(" "), padding=True, truncation=True, return_tensors ="pt")
    with torch.no_grad():
      outputs = model(**tokenized_inputs)
      predictions = torch.argmax(outputs.logits, dim=1)

      predicted_labels = ["Mountain" if pred > 0.7 else "." for pred in predictions.tolist()]
      return predicted_labels


sentences_to_predict = [
    "expedition to Ukraine is cool, i love Svydovec, Alps"

    ]


for sentence_x in sentences_to_predict:
  predicted_labels =  predict_sentence(sentence_x, model, tokenizer)
  for i in zip(sentence_x.split(" "), predicted_labels):
    print(f"{i[0]}({i[1]}) ", end="")
  print("")


expedition(.) to(.) Ukraine(.) is(.) cool,(.) i(.) love(.) Svydovec,(.) Alps(.) 


In [ ]:
i = 0
while True:
  print(f"slept well {i}")
  time.sleep(5)
  i+= 5


slept well 0
slept well 5
slept well 10
slept well 15
slept well 20
slept well 25
slept well 30
slept well 35
slept well 40
slept well 45
slept well 50
slept well 55
slept well 60
slept well 65
slept well 70
slept well 75
slept well 80
slept well 85
slept well 90
slept well 95
slept well 100


KeyboardInterrupt: ignored